In [8]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
from time import sleep # background color fade
import math # calculo com sin e cos

glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(720, 600, "Exercicio 3 by Joao Villaca", None, None)
glfw.make_context_current(window)

vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")
    
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)


num_vertices = 5

vertices = np.zeros(num_vertices, [("position", np.float32, 3)])
vertices['position'] = [
    # Face 1 do Cubo (vértices do quadrado)
    (-0.2, -0.2, +0.2),
    (+0.2, -0.2, +0.2),
    (-0.2, +0.2, +0.2),
    (+0.2, +0.2, +0.2),
    (+0.15, +0.15, +0.0)]

#print(vertices) # debug

# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

loc_color = glGetUniformLocation(program, "color")    

x_var = 0.0
y_var = 0.0

def key_listener(window, key, scancode, action, mods):
    
#    print('[key event] key=',key)
#    print('[key event] scancode=',scancode)
#    print('[key event] action=',action)
#    print('[key event] mods=',mods)
#    print('-------')

    global x_var, y_var
    
    # comandos de rotação
    if key == 262 and action > 0: x_var += 0.1
    if key == 263 and action > 0: x_var -= 0.1
    if key == 264 and action > 0: y_var += 0.1
    if key == 265 and action > 0: y_var -= 0.1

z_var = 0.0
def mouse_listener(window, button, action, mods):
    
#    print('[mouse event] button=',button)
#    print('[mouse event] action=',action)
#    print('[mouse event] mods=',mods)
#    print('-------')   

    global z_var
    if button == 0:
        z_var += 0.1
    if button == 1:
        z_var -= 0.1

glfw.set_mouse_button_callback(window, mouse_listener)
glfw.set_key_callback(window, key_listener)
glfw.show_window(window)
glEnable(GL_DEPTH_TEST) ### importante para 3D

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

R = 0.0
G = 0.0
B = 0.0

while not glfw.window_should_close(window):

    glfw.poll_events()
    
    #efeito RGB em loop
    if B < 0.03:
        switcher = 0
    if B > 0.99:
        switcher = 1

    sleep(0.01)
    
    if switcher == 0:
        R = R + 0.03
        G = G + 0.01
        B = B + 0.01
    else:
        R = R - 0.03
        G = G - 0.01
        B = B - 0.01
    
    
    cos_x_var = math.cos(x_var)
    sin_x_var = math.sin(x_var)
    cos_y_var = math.cos(y_var)
    sin_y_var = math.sin(y_var)
    cos_z_var = math.cos(z_var)
    sin_z_var = math.sin(z_var)
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(0, 0, 0, 1)
    
    mat_rotation_z = np.array([     cos_z_var, -sin_z_var, 0.0, 0.0, 
                                    sin_z_var,  cos_z_var, 0.0, 0.0, 
                                    0.0,      0.0, 1.0, 0.0, 
                                    0.0,      0.0, 0.0, 1.0], np.float32)
    
    mat_rotation_x = np.array([     1.0,   0.0,    0.0, 0.0, 
                                    0.0, cos_x_var, -sin_x_var, 0.0, 
                                    0.0, sin_x_var,  cos_x_var, 0.0, 
                                    0.0,   0.0,    0.0, 1.0], np.float32)
    
    mat_rotation_y = np.array([     cos_y_var,  0.0, sin_y_var, 0.0, 
                                    0.0,    1.0,   0.0, 0.0, 
                                    -sin_y_var, 0.0, cos_y_var, 0.0, 
                                    0.0,    0.0,   0.0, 1.0], np.float32)
    
    mat_transform = multiplica_matriz(mat_rotation_x,mat_rotation_y)
    mat_transform = multiplica_matriz(mat_transform,mat_rotation_z)


    glUniform4f(loc_color, R, G, B, 1.0);
    glDrawArrays(GL_TRIANGLE_STRIP, 0, num_vertices)

    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)

    glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)
    
    glfw.swap_buffers(window)

glfw.terminate()